> # 5 오차역전파법

## 5.1 계산 그래프

- 노드(node)
- 에지(edge): 노드 사이의 직선

### 5.1.1 계산 그래프로 풀다

① 계산 그래프를 구성한다.

② 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. (순전파)

### 5.1.2 국소적 계산

- 국소적이란 '자신과 직접 관계된 작은 범위'라는 뜻
- 국소적 계산은 전체에서 자신과 관계된 정보만으로 결과를 출력

### 5.1.3 왜 계산 그래프로 푸는가?

- 역전파를 통해 '미분'을효율적으로 계산할 수 있음

## 5.2 연쇄법칙

### 5.2.1 계산 그래프의 역전파

- 순방향과는 반대 방향으로 국소적 미분을 곱한다.

### 5.2.2 연쇄법칙이란?

#### 합성 함수

- 다음의 z 처럼 여러 함수로 구성된 함수

## <center> $z = (x + y)^2$ </center>

## <center> $z = t^2$ </center>

## <center> $t = x + y$ </center>

- 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있음

## <center>$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$ </center>

### 5.2.3 연쇄법칙과 계산 그래프

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

- 덧셈 노드의 역전파는 입력 값을 그대로 흘려보냄(1을 곱해주는 것이기 때문)
- $z = x + y$라는 식에서의 미분은 다음과 같음

## <center>$\frac{\partial z}{\partial x} = 1$ </center>

## <center>$\frac{\partial z}{\partial y} = 1$ </center>

### 5.3.2 곱셈 노드의 역전파

- 곱셈 노드의 역전파는 '순전파 때의 입력 신호들을 서로 바꾼 값'을 곱해줌
- $z = xy$라는 식에서의 미분은 다음과 같음

## <center>$\frac{\partial z}{\partial x} = y$ </center>

## <center>$\frac{\partial z}{\partial y} = x$ </center>

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [12]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

In [13]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [14]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [15]:
dapple_price, dtax

(1.1, 200)

### 5.4.2 덧셈 계층

In [6]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [9]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층